## Note that this notebook should be run in GCP Colab environment

Runtime requirements:
- min. 750 GB disk space
- min. 500 GB RAM

In [ ]:
!pwd

/content


In [ ]:
!git clone https://github.com/EMBL-EBI-ABC/PerturbationCatalogue.git

Cloning into 'PerturbationCatalogue'...
remote: Enumerating objects: 4128, done.
remote: Counting objects: 100% (1326/1326), done.
remote: Compressing objects: 100% (602/602), done.
remote: Total 4128 (delta 822), reused 798 (delta 724), pack-reused 2802 (from 2)
Receiving objects: 100% (4128/4128), 58.79 MiB | 46.74 MiB/s, done.
Resolving deltas: 100% (2728/2728), done.


In [ ]:
%cd PerturbationCatalogue/data_exploration/

/content/PerturbationCatalogue/data_exploration


In [ ]:
!git pull
!git switch aleks/bq_table

remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 11 (delta 8), reused 11 (delta 8), pack-reused 0 (from 0)
Unpacking objects: 100% (11/11), 4.34 KiB | 2.00 KiB/s, done.
From https://github.com/EMBL-EBI-ABC/PerturbationCatalogue
   769b40f..827153c  aleks/curate_nadig_2025 -> origin/aleks/curate_nadig_2025
   62127cc..4019a01  dev                     -> origin/dev
Updating 769b40f..827153c
error: Your local changes to the following files would be overwritten by merge:
	data_exploration/curation_tools/ontologies/gene_ont.parquet
Please commit your changes or stash them before you merge.
Aborting
error: Your local changes to the following files would be overwritten by checkout:
	data_exploration/curation_tools/ontologies/gene_ont.parquet
	data_exploration/local_curation_tools.egg-info/SOURCES.txt
Please commit your changes or stash them before you switch branches.
Aborting


In [ ]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.0/627.0 kB 89.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of umap-learn to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.9/169.9 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━

In [ ]:
!gcloud init --project "prj-ext-dev-pertcat-437314" --account "zakirov@ebi.ac.uk"

Welcome! This command will take you through the configuration of gcloud.

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

You are signed in as: [zakirov@ebi.ac.uk].

Your current project has been set to: [prj-ext-dev-pertcat-437314].

Do you want to configure a default Compute Region and Zone? (Y/n)?  n

The Google Cloud CLI is configured and ready to use!

* Commands that require authentication will use zakirov@ebi.ac.uk by default
* Commands will reference project `prj-ext-dev-pertcat-437314` by default
Run `gcloud help config` to learn how to change individual settings

This gcloud configuration is called [default]. You can create additional configurations if you work with multiple accounts and/or projects.
Run `gcloud topic configuratio

In [ ]:
!mkdir -p Perturbseq/non_curated/h5ad/

In [ ]:
!gcloud storage cp gs://perturbation-catalogue-lake/perturbseq/non_curated/orion_2025_hct116.h5ad Perturbseq/non_curated/h5ad/

Copying gs://perturbation-catalogue-lake/perturbseq/non_curated/orion_2025_hct116.h5ad to file://Perturbseq/non_curated/h5ad/orion_2025_hct116.h5ad


Command killed by keyboard interrupt



Command killed by keyboard interrupt



Command killed by keyboard interrupt



Command killed by keyboard interrupt



Command killed by keyboard interrupt



Command killed by keyboard interrupt



Command killed by keyboard interrupt



Command killed by keyboard interrupt



Command killed by keyboard interrupt


Average throughput: 882.8MiB/s
ERROR: gcloud crashed (RuntimeError): release unlocked lock

If you would like to report this issue, please run the following command:
  gcloud feedback

To check gcloud for common problems, please run the following command:
  gcloud info --run-diagnostics
^C


# Import

In [ ]:
import pandas as pd
import json

from curation_tools.curation_tools import (
    CuratedDataset,
    ObsSchema,
    VarSchema,
    Experiment,
    download_file,
    upload_parquet_to_bq
)

# import logging
# logging.basicConfig(
#     level=logging.DEBUG,
#     format="%(asctime)s %(levelname)s %(name)s: %(message)s",
#     handlers=[
#         logging.FileHandler("curation.log"),
#         logging.StreamHandler(),  # keep console output too
#     ],
#     force=True,
# )

/usr/local/lib/python3.12/dist-packages/pandera/_pandas_deprecated.py:157: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


# Pre-process the data

### Load the data and construct the dummy anndata object to explore the dataset offline

In [ ]:
# from scipy.sparse import csr_matrix
# import numpy as np
# import scanpy as sc

# N1 = obs.shape[0]  # Number of objects (e.g., cells)
# N2 = var.shape[0]  # Number of categories (e.g., genes)
# rows = np.arange(N1)
# cols = (np.floor(np.random.permutation(N1)/float(N1)*N2)).astype(int) # Randomly pick N1 objects and assign to N2 categories in almost equal proportion
# w = np.ones(N1)
# conn_matrix = csr_matrix((w, (rows, cols)), shape=(N1, N2), dtype=int)

# adata = sc.AnnData(X=conn_matrix, obs=obs, var=var)

# # save the non-curated AnnData object
# adata.write_h5ad("Perturbseq/non_curated/h5ad/orion_2025_dummy.h5ad")

# Initialise the dataset object

In [ ]:
noncurated_path = "Perturbseq/non_curated/h5ad/orion_2025_hct116.h5ad"
cur_data = CuratedDataset(
    obs_schema=ObsSchema,
    var_schema=VarSchema,
    exp_metadata_schema=Experiment,
    noncurated_path=noncurated_path
)

cur_data.load_data()

Loading data from Perturbseq/non_curated/h5ad/orion_2025_hct116.h5ad


In [ ]:
cur_data.adata.obs

,sample,num_features,guide_target,gene_target,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,pass_guide_filter
AAACCAAAGACATGTT-HCT116_Batch1,HCT116_Batch1,2,ST14_P1P2-1|ST14_P1P2-2,ST14,4883,19136.0,1179.0,6.161162,True
AAACCAAAGACCCAAC-HCT116_Batch1,HCT116_Batch1,2,SIGLEC5_P1P2-1|SIGLEC5_P1P2-2,SIGLEC5,8130,47916.0,1562.0,3.259871,True
AAACCAAAGAGGTACG-HCT116_Batch1,HCT116_Batch1,2,VSNL1_P1P2-1|VSNL1_P1P2-2,VSNL1,6531,28435.0,1042.0,3.664498,True
AAACCAAAGCGATTAT-HCT116_Batch1,HCT116_Batch1,2,KCNK7_P1P2-1|KCNK7_P1P2-2,KCNK7,5931,26080.0,1087.0,4.167945,True
AAACCAAAGGCTTAAT-HCT116_Batch1,HCT116_Batch1,2,APOA4_P1P2-1|APOA4_P1P2-2,APOA4,7157,38366.0,955.0,2.489183,True
...,...,...,...,...,...,...,...,...,...
GTTGTGCAGGGAATCG-HCT116_Batch109,HCT116_Batch109,2,ACP6_P1P2-1|ACP6_P1P2-2,ACP6,5107,17882.0,767.0,4.289229,True
GTTGTGGGTCACGCGG-HCT116_Batch109,HCT116_Batch109,2,RNF39_P1P2-1|RNF39_P1P2-2,RNF39,3643,8155.0,664.0,8.142244,True
GTTGTGGGTCTGACTC-HCT116_Batch109,HCT116_Batch109,2,AFP_P2-1|AFP_P2-2,AFP,2831,6334.0,1553.0,24.518473,True
GTTGTGGGTCTTGATT-HCT116_Batch109,HCT116_Batch109,2,METTL9_P1P2-1|METTL9_P1P2-2,METTL9,3868,8766.0,868.0,9.901894,True


In [ ]:
cur_data.adata.var['gene_symbol'] = cur_data.adata.var.index
cur_data.show_var()

Variable data:
DataFrame shape: (38606, 6)
--------------------------------------------------
                    mt  n_cells_by_counts   mean_counts  \
DDX11L2          False               2719  2.777434e-04   
MIR1302-2HG      False               2620  2.673255e-04   
FAM138A          False                  3  3.034341e-07   
ENSG00000290826  False                764  8.698444e-05   
OR4F5            False                263  2.700564e-05   
...                ...                ...           ...   
ENSG00000277836  False                  0  0.000000e+00   
ENSG00000278633  False                106  1.092363e-05   
ENSG00000276017  False                663  6.756467e-05   
ENSG00000278817  False              35369  3.616227e-03   
ENSG00000277196  False              48372  5.025274e-03   

                 pct_dropout_by_counts  total_counts      gene_symbol  
DDX11L2                      99.972499        2746.0          DDX11L2  
MIR1302-2HG                  99.973500        2643.0 

# OBS slot curation

In [ ]:
cur_data.adata.obs.isna().sum()

,0
sample,0
num_features,0
guide_target,0
gene_target,0
n_genes_by_counts,0
total_counts,0
total_counts_mt,0
pct_counts_mt,0
pass_guide_filter,0


### Extract perturbation symbols

### Rename `guide_target` to `perturbation_name`

In [ ]:
cur_data.rename_columns(slot = 'obs', name_dict = {'guide_target': 'perturbation_name'})

Renamed columns in adata.obs: {'guide_target': 'perturbation_name'}


### Rename `gene_target` to `perturbed_target_symbol`

In [ ]:
cur_data.rename_columns(slot = 'obs', name_dict = {'gene_target': 'perturbed_target_symbol'})

Renamed columns in adata.obs: {'gene_target': 'perturbed_target_symbol'}


#### Clean up `perturbed_target_symbol` column

In [ ]:
cur_data.replace_entries(
    slot="obs",
    column="perturbed_target_symbol",
    map_dict={
        "Non-Targeting": "control_nontargeting"
    }
)

cur_data.show_unique(slot = 'obs', column = 'perturbed_target_symbol')

Streaming output truncated to the last 5000 lines.
 'RP1L1',
 'RP2',
 'RP9',
 'RPA1',
 'RPA2',
 'RPA3',
 'RPA4',
 'RPAIN',
 'RPAP1',
 'RPAP2',
 'RPAP3',
 'RPE',
 'RPE65',
 'RPF1',
 'RPF2',
 'RPGR',
 'RPGRIP1',
 'RPGRIP1L',
 'RPH3A',
 'RPH3AL',
 'RPIA',
 'RPL10',
 'RPL10A',
 'RPL10L',
 'RPL11',
 'RPL12',
 'RPL13',
 'RPL13A',
 'RPL14',
 'RPL15',
 'RPL17',
 'RPL18',
 'RPL18A',
 'RPL19',
 'RPL21',
 'RPL22',
 'RPL22L1',
 'RPL23',
 'RPL23A',
 'RPL24',
 'RPL26',
 'RPL26L1',
 'RPL27',
 'RPL27A',
 'RPL28',
 'RPL29',
 'RPL3',
 'RPL30',
 'RPL31',
 'RPL32',
 'RPL34',
 'RPL35',
 'RPL35A',
 'RPL36',
 'RPL36A',
 'RPL36AL',
 'RPL37',
 'RPL37A',
 'RPL38',
 'RPL39',
 'RPL39L',
 'RPL3L',
 'RPL4',
 'RPL41',
 'RPL5',
 'RPL6',
 'RPL7',
 'RPL7A',
 'RPL7L1',
 'RPL8',
 'RPL9',
 'RPLP0',
 'RPLP1',
 'RPLP2',
 'RPN1',
 'RPN2',
 'RPP14',
 'RPP21',
 'RPP25',
 'RPP25L',
 'RPP30',
 'RPP40',
 'RPRD1A',
 'RPRD1B',
 'RPRD2',
 'RPRM',
 'RPRML',
 'RPS10',
 'RPS11',
 'RPS12',
 'RPS13',
 'RPS14',
 'RPS15',
 'RPS15A',
 'RPS1

### Standardise perturbation targets

In [ ]:
cur_data.standardize_genes(
    slot='obs',
    input_column='perturbed_target_symbol',
    input_column_type='gene_symbol',
    multiple_entries=False
)

Mapping gene symbols: 100%|████████████████████████████████| 18294/18294 [00:00<00:00, 25285.63it/s]


--------------------------------------------------
Successfully mapped 18288 out of 18294 gene symbols.
--------------------------------------------------
Couldn't map gene symbols: ['control_nontargeting', 'PRY2', 'PRY', 'LINC02911', 'SMIM2', 'LINC02913']
--------------------------------------------------


### Add `perturbed_target_number` column

In [ ]:
cur_data.adata.obs = cur_data.adata.obs.iloc[:,[0,1,2,3,4,5,6,7,8,9,11,12,13]]

In [ ]:
cur_data.count_entries(
    slot='obs',
    input_column='perturbed_target_symbol',
    count_column_name='perturbed_target_number',
    sep='|'
)

Counted entries in column perturbed_target_symbol of adata.obs and stored in perturbed_target_number


### Encode chromosomes as integers

In [ ]:
cur_data.chromosome_encoding()

Chromosome encoding applied to perturbed_target_chromosome in adata.obs and stored as 'perturbed_target_chromosome_encoding'.


In [ ]:
cur_data.show_obs(['perturbation_name', 'perturbed_target_chromosome_encoding'])

Observation data:
DataFrame shape: (3409169, 2)
--------------------------------------------------
                                              perturbation_name  \
index                                                             
AAACCAAAGACATGTT-HCT116_Batch1          ST14_P1P2-1|ST14_P1P2-2   
AAACCAAAGACCCAAC-HCT116_Batch1    SIGLEC5_P1P2-1|SIGLEC5_P1P2-2   
AAACCAAAGAGGTACG-HCT116_Batch1        VSNL1_P1P2-1|VSNL1_P1P2-2   
AAACCAAAGCGATTAT-HCT116_Batch1        KCNK7_P1P2-1|KCNK7_P1P2-2   
AAACCAAAGGCTTAAT-HCT116_Batch1        APOA4_P1P2-1|APOA4_P1P2-2   
...                                                         ...   
GTTGTGCAGGGAATCG-HCT116_Batch109        ACP6_P1P2-1|ACP6_P1P2-2   
GTTGTGGGTCACGCGG-HCT116_Batch109      RNF39_P1P2-1|RNF39_P1P2-2   
GTTGTGGGTCTGACTC-HCT116_Batch109              AFP_P2-1|AFP_P2-2   
GTTGTGGGTCTTGATT-HCT116_Batch109    METTL9_P1P2-1|METTL9_P1P2-2   
GTTGTGGGTGTAGTTA-HCT116_Batch109  SLC5A10_P1P2-1|SLC5A10_P1P2-2   

                             

In [ ]:
cur_data.adata.obs.head()

,pct_counts_mt,pass_guide_filter,num_features,total_counts_mt,total_counts,sample,n_genes_by_counts,perturbation_name,perturbed_target_ensg,perturbed_target_symbol,perturbed_target_biotype,perturbed_target_coord,perturbed_target_chromosome,perturbed_target_number,perturbed_target_chromosome_encoding
index,,,,,,,,,,,,,,,
AAACCAAAGACATGTT-HCT116_Batch1,6.161162,True,2,1179.0,19136.0,HCT116_Batch1,4883,ST14_P1P2-1|ST14_P1P2-2,ENSG00000149418,ST14,protein_coding,chr11:130159746-130210367;1,11,1,11
AAACCAAAGACCCAAC-HCT116_Batch1,3.259871,True,2,1562.0,47916.0,HCT116_Batch1,8130,SIGLEC5_P1P2-1|SIGLEC5_P1P2-2,ENSG00000105501,SIGLEC5,lncRNA,chr19:51630101-51645545;-1,19,1,19
AAACCAAAGAGGTACG-HCT116_Batch1,3.664498,True,2,1042.0,28435.0,HCT116_Batch1,6531,VSNL1_P1P2-1|VSNL1_P1P2-2,ENSG00000163032,VSNL1,protein_coding,chr2:17539020-17657018;1,2,1,2
AAACCAAAGCGATTAT-HCT116_Batch1,4.167945,True,2,1087.0,26080.0,HCT116_Batch1,5931,KCNK7_P1P2-1|KCNK7_P1P2-2,ENSG00000173338,KCNK7,protein_coding,chr11:65592836-65595996;-1,11,1,11
AAACCAAAGGCTTAAT-HCT116_Batch1,2.489183,True,2,955.0,38366.0,HCT116_Batch1,7157,APOA4_P1P2-1|APOA4_P1P2-2,ENSG00000110244,APOA4,protein_coding,chr11:116820700-116823304;-1,11,1,11


### Add guide RNA information

The original pre-print does not contain information about guide RNAs in the `obs` slot.

However, the pre-print references the source of the library - [Maximizing CRISPRi efficacy and accessibility with dual-sgRNA libraries and optimal effectors. Replogle et al 2022](https://doi.org/10.7554/eLife.81856).

Supplementary table s4 contains the information about guide RNAs used in the experiment:

Supplementary table S4: https://elifesciences.org/download/aHR0cHM6Ly9jZG4uZWxpZmVzY2llbmNlcy5vcmcvYXJ0aWNsZXMvODE4NTYvZWxpZmUtODE4NTYtc3VwcDQtdjIueGxzeA--/elife-81856-supp4-v2.xlsx?_hash=AmEXp%2BOl1z1Y4JCBAaVhST5ImQmKAGdfEjGhuJbSJ9Q%3D

In [ ]:
# get synonyms from the gene ontology
syn_df = cur_data.gene_ont[["synonym", "gene_symbol"]].drop_duplicates()
syn_df["gene_symbol"] = syn_df["gene_symbol"].fillna("")
syn_df["synonym"] = syn_df["synonym"].fillna("")
syn_df["all_genes"] = (
    (syn_df["gene_symbol"] + "|" + syn_df["synonym"]).str.lstrip("|").str.rstrip("|")
)
syn_df["all_genes_match"] = syn_df["all_genes"].str.split("|")
syn_df = syn_df.explode("all_genes_match")
syn_df["all_genes_match"] = syn_df["all_genes_match"].str.upper()

# read data from supplementary files
guide_ids_df = pd.read_csv(
    "Perturbseq/supplementary/orion_2025_weissman_dual_crispri_sgrnas.csv"
)
guide_ids_df["gene"] = guide_ids_df["gene"].str.upper()

guide_ids_df = guide_ids_df.merge(
    syn_df, left_on="gene", right_on="all_genes_match", how="left"
)

guide_ids_df["all_genes"] = guide_ids_df["all_genes"].str.split("|")
guide_ids_df = guide_ids_df.explode("all_genes").drop_duplicates()

# create a perturbation name column
guide_ids_df["perturbation_name"] = (
    guide_ids_df["all_genes"]
    + "_"
    + guide_ids_df["transcript"]
    + "|"
    + guide_ids_df["all_genes"]
    + "_"
    + guide_ids_df["transcript"]
)

guide_ids_df["perturbation_name"] = guide_ids_df["perturbation_name"].str.upper()

# replace commas with underscores in the perturbation name
guide_ids_df["perturbation_name"] = guide_ids_df["perturbation_name"].str.replace(
    ",", "_"
)

# create a dual guide RNA sequence column
guide_ids_df["dual_guide_sequence"] = (
    guide_ids_df["protospacer_A"] + "|" + guide_ids_df["protospacer_B"]
).str.upper()

guide_ids_df.loc[
    guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "perturbation_name"
] = (
    guide_ids_df.loc[
        guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "gene"
    ]
    + "_"
    + guide_ids_df.loc[
        guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "transcript"
    ]
    + "|"
    + guide_ids_df.loc[
        guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "gene"
    ]
    + "_"
    + guide_ids_df.loc[
        guide_ids_df["all_genes"].isna() & guide_ids_df["gene"].notna(), "transcript"
    ]
)

# replace NEGATIVE_CONTROL with actual non targeting guide names
guide_ids_df.loc[guide_ids_df["gene"] == "NEGATIVE_CONTROL", "perturbation_name"] = (
    guide_ids_df.loc[guide_ids_df["gene"] == "NEGATIVE_CONTROL", "sgID_AB"].str.upper()
)

# create dictionaries for guide sequences mapping
guide_ids_dict = (
    guide_ids_df[["perturbation_name", "dual_guide_sequence"]]
    .dropna()
    .drop_duplicates()
    .set_index("perturbation_name")["dual_guide_sequence"]
    .to_dict()
)

# add guide sequences to the obs dataframe
obj_df = cur_data.adata.obs.copy()

# remove the numeric suffices from the perturbation names
obj_df["perturbation_name_matching"] = obj_df["perturbation_name"].str.replace(
    r"-\d(?=\||$)", "", regex=True
)
obj_df["perturbation_name_matching"] = obj_df["perturbation_name_matching"].str.upper()

# map the guide sequences to the perturbation names
obj_df["guide_sequence"] = obj_df["perturbation_name_matching"].map(guide_ids_dict)

obj_df = obj_df[["guide_sequence"]]

obj_df

,guide_sequence
index,
AAACCAAAGACATGTT-HCT116_Batch1,GGCCCAGCGCGCCCGACCGG|GAGAGAGCGCGCCAGGGCGA
AAACCAAAGACCCAAC-HCT116_Batch1,GCCCGAGGCCCTGACTCCCA|GTATGGGACCTGGTGACCTC
AAACCAAAGAGGTACG-HCT116_Batch1,GTGCAAGGCGCGATCCAAGA|GGATTTACCCGAGCGTGTTG
AAACCAAAGCGATTAT-HCT116_Batch1,GAAGGCCCTGGTCCCGATAC|GAGTTCCCACAGCACCCCAG
AAACCAAAGGCTTAAT-HCT116_Batch1,GGCGCAGGTGAGCTCTCCTG|GTTGTAGGGAGGATCCAGTG
...,...
GTTGTGCAGGGAATCG-HCT116_Batch109,GAGACCCCGAGTGGGAACGG|GCCTGAGGGAGACCCCGAGT
GTTGTGGGTCACGCGG-HCT116_Batch109,GTGAAGAGAGAGGCAATCCC|GGGCAATCCCAGGAGAGGGG
GTTGTGGGTCTGACTC-HCT116_Batch109,GTCCGACGAGGTCAGCTGGG|GATGGCCATCAGCTCCGACG


In [ ]:
# merge guide sequences with main
cur_data.adata.obs = cur_data.adata.obs.merge(
    obj_df,
    left_index=True,
    right_index=True,
    how="left",
)

cur_data.adata.obs

,pct_counts_mt,pass_guide_filter,num_features,total_counts_mt,total_counts,sample,n_genes_by_counts,perturbation_name,perturbed_target_ensg,perturbed_target_symbol,perturbed_target_biotype,perturbed_target_coord,perturbed_target_chromosome,perturbed_target_number,perturbed_target_chromosome_encoding,guide_sequence
index,,,,,,,,,,,,,,,,
AAACCAAAGACATGTT-HCT116_Batch1,6.161162,True,2,1179.0,19136.0,HCT116_Batch1,4883,ST14_P1P2-1|ST14_P1P2-2,ENSG00000149418,ST14,protein_coding,chr11:130159746-130210367;1,11,1,11,GGCCCAGCGCGCCCGACCGG|GAGAGAGCGCGCCAGGGCGA
AAACCAAAGACCCAAC-HCT116_Batch1,3.259871,True,2,1562.0,47916.0,HCT116_Batch1,8130,SIGLEC5_P1P2-1|SIGLEC5_P1P2-2,ENSG00000105501,SIGLEC5,lncRNA,chr19:51630101-51645545;-1,19,1,19,GCCCGAGGCCCTGACTCCCA|GTATGGGACCTGGTGACCTC
AAACCAAAGAGGTACG-HCT116_Batch1,3.664498,True,2,1042.0,28435.0,HCT116_Batch1,6531,VSNL1_P1P2-1|VSNL1_P1P2-2,ENSG00000163032,VSNL1,protein_coding,chr2:17539020-17657018;1,2,1,2,GTGCAAGGCGCGATCCAAGA|GGATTTACCCGAGCGTGTTG
AAACCAAAGCGATTAT-HCT116_Batch1,4.167945,True,2,1087.0,26080.0,HCT116_Batch1,5931,KCNK7_P1P2-1|KCNK7_P1P2-2,ENSG00000173338,KCNK7,protein_coding,chr11:65592836-65595996;-1,11,1,11,GAAGGCCCTGGTCCCGATAC|GAGTTCCCACAGCACCCCAG
AAACCAAAGGCTTAAT-HCT116_Batch1,2.489183,True,2,955.0,38366.0,HCT116_Batch1,7157,APOA4_P1P2-1|APOA4_P1P2-2,ENSG00000110244,APOA4,protein_coding,chr11:116820700-116823304;-1,11,1,11,GGCGCAGGTGAGCTCTCCTG|GTTGTAGGGAGGATCCAGTG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTTGTGCAGGGAATCG-HCT116_Batch109,4.289229,True,2,767.0,17882.0,HCT116_Batch109,5107,ACP6_P1P2-1|ACP6_P1P2-2,ENSG00000162836,ACP6,protein_coding,chr1:147629652-147670531;-1,1,1,1,GAGACCCCGAGTGGGAACGG|GCCTGAGGGAGACCCCGAGT
GTTGTGGGTCACGCGG-HCT116_Batch109,8.142244,True,2,664.0,8155.0,HCT116_Batch109,3643,RNF39_P1P2-1|RNF39_P1P2-2,ENSG00000204618,RNF39,protein_coding,chr6:30070266-30075849;-1,6,1,6,GTGAAGAGAGAGGCAATCCC|GGGCAATCCCAGGAGAGGGG
GTTGTGGGTCTGACTC-HCT116_Batch109,24.518473,True,2,1553.0,6334.0,HCT116_Batch109,2831,AFP_P2-1|AFP_P2-2,ENSG00000081051,AFP,protein_coding,chr4:73431138-73456174;1,4,1,4,GTCCGACGAGGTCAGCTGGG|GATGGCCATCAGCTCCGACG


### Add metadata

In [ ]:
cur_data.create_columns(
    slot="obs",
    col_dict={
        "dataset_id": cur_data.dataset_id,
        "sample_id": range(1, cur_data.adata.obs.shape[0] + 1),
        # treatment
        "treatment_label": None,
        "treatment_id": None,
        # perturbation type
        "perturbation_type_label": "CRISPRi",
        "perturbation_type_id": None,
        # model system
        "model_system_label": "cell_line",
        "model_system_id": None,

        "data_modality": "Perturb-seq",
        "significant": None,
        "significance_criteria": None,
        "score_interpretation": None,

        # replicates
        "technical_replicate": None,
        "biological_replicate": None,

        'tissue': 'colon',
        'cell_type': 'colon epithelial cell',
        'cell_line': 'HCT 116 cell',
        "timepoint": "P0DT0H0M0S",
        'disease': 'colon carcinoma',

        "species": "Homo sapiens",
        "sex_label": "male",
        "sex_id": None,
        "developmental_stage_label": "adult",
        "developmental_stage_id": None,

        "study_title": "X-Atlas/Orion: Genome-wide Perturb-seq Datasets via a Scalable Fix-Cryopreserve Platform for Training Dose-Dependent Biological Foundation Models",
        "study_uri": "https://doi.org/10.1101/2025.06.11.659105",
        "study_year": 2025,
        "first_author": "Ann Huang",
        "last_author": "Ci Chu",

        "experiment_title": "Over 3.4 million HCT116 cells were transfected with dual gRNAs targeting the same gene, and whole genome CRISPRi Perturb-seq was performed.",
        "experiment_summary": "The researchers developed the Fix-Cryopreserve-ScRNAseq (FiCS) Perturb-seq platform to enable scalable and reproducible genome-wide perturbation screening in human cell lines. Using this platform, they generated a genome-wide dual-guide CRISPRi Perturb-seq dataset in HCT116 colon carcinoma cells. The dataset includes over 3.4 million deeply sequenced cells, with each cell yielding over 16,000 unique molecular identifiers (UMIs).",
        "number_of_perturbed_targets": len(set(cur_data.adata.obs['perturbed_target_coord'])),
        "number_of_perturbed_samples": cur_data.adata.obs.shape[0],

        "library_generation_type_id": "EFO:0022868",
        "library_generation_type_label": "endogenous",

        "library_generation_method_id": "EFO:0022895",
        "library_generation_method_label": "dCas9-KRAB",

        "enzyme_delivery_method_id": None,
        "enzyme_delivery_method_label": "nanoparticle-mediated transfection",

        "library_delivery_method_id": None,
        "library_delivery_method_label": "lentivirus transduction",

        "enzyme_integration_state_id": None,
        "enzyme_integration_state_label": "random locus integration",

        "library_integration_state_id": None,
        "library_integration_state_label": "random locus integration",

        "enzyme_expression_control_id": None,
        "enzyme_expression_control_label": "constitutive transgene expression",

        "library_expression_control_id": None,
        "library_expression_control_label": "constitutive transgene expression",

        "library_name": "custom",
        "library_uri": None,

        "library_format_id": None,
        "library_format_label": "pooled",

        "library_scope_id": None,
        "library_scope_label": "genome-wide",

        "library_perturbation_type_id": None,
        "library_perturbation_type_label": "inhibition",

        "library_manufacturer": "Weissman",
        "library_lentiviral_generation": "3",
        "library_grnas_per_target": "2",
        "library_total_grnas": "20845",
        "library_total_variants": None,

        "readout_dimensionality_id": None,
        "readout_dimensionality_label": "high-dimensional assay",

        "readout_type_id": None,
        "readout_type_label": "transcriptomic",

        "readout_technology_id": None,
        "readout_technology_label": "single-cell rna-seq",

        "method_name_id": None,
        "method_name_label": "Perturb-seq",

        "method_uri": None,

        "sequencing_library_kit_id": None,
        "sequencing_library_kit_label": "10x Genomics Chromium GEM-X Single Cell 5-prime kit v3",

        "sequencing_platform_id": None,
        "sequencing_platform_label": "Illumina NovaSeq X Plus",

        "sequencing_strategy_id": None,
        "sequencing_strategy_label": "barcode sequencing",

        "software_counts_id": None,
        "software_counts_label": "CellRanger",

        "software_analysis_id": None,
        "software_analysis_label": "custom",

        "reference_genome_id": None,
        "reference_genome_label": "GRCh38",

        "license_label": "CC BY-NC-SA 4.0",
        "license_id": "SWO:1000090",

        "associated_datasets": json.dumps([
            {
                "dataset_accession": "HCT116_filtered_dual_guide_cells",
                "dataset_uri": "https://plus.figshare.com/ndownloader/files/55021257",
                "dataset_description": "Filtered raw counts",
                "dataset_file_name": "HCT116_filtered_dual_guide_cells.h5ad",
            }
        ])
    }
)

Column dataset_id added to adata.obs
Column sample_id added to adata.obs
Column treatment_label added to adata.obs
Column treatment_id added to adata.obs
Column perturbation_type_label added to adata.obs
Column perturbation_type_id added to adata.obs
Column model_system_label added to adata.obs
Column model_system_id added to adata.obs
Column data_modality added to adata.obs
Column significant added to adata.obs
Column significance_criteria added to adata.obs
Column score_interpretation added to adata.obs
Column technical_replicate added to adata.obs
Column biological_replicate added to adata.obs
Column tissue added to adata.obs
Column cell_type added to adata.obs
Column cell_line added to adata.obs
Column timepoint added to adata.obs
Column disease added to adata.obs
Column species added to adata.obs
Column sex_label added to adata.obs
Column sex_id added to adata.obs
Column developmental_stage_label added to adata.obs
Column developmental_stage_id added to adata.obs
Column study_titl

In [ ]:
cur_data.adata.obs

,pct_counts_mt,pass_guide_filter,num_features,total_counts_mt,total_counts,sample,n_genes_by_counts,perturbation_name,perturbed_target_ensg,perturbed_target_symbol,...,sequencing_strategy_label,software_counts_id,software_counts_label,software_analysis_id,software_analysis_label,reference_genome_id,reference_genome_label,license_label,license_id,associated_datasets
index,,,,,,,,,,,,,,,,,,,,,
AAACCAAAGACATGTT-HCT116_Batch1,6.161162,True,2,1179.0,19136.0,HCT116_Batch1,4883,ST14_P1P2-1|ST14_P1P2-2,ENSG00000149418,ST14,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HCT116_filtered_dual_g..."
AAACCAAAGACCCAAC-HCT116_Batch1,3.259871,True,2,1562.0,47916.0,HCT116_Batch1,8130,SIGLEC5_P1P2-1|SIGLEC5_P1P2-2,ENSG00000105501,SIGLEC5,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HCT116_filtered_dual_g..."
AAACCAAAGAGGTACG-HCT116_Batch1,3.664498,True,2,1042.0,28435.0,HCT116_Batch1,6531,VSNL1_P1P2-1|VSNL1_P1P2-2,ENSG00000163032,VSNL1,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HCT116_filtered_dual_g..."
AAACCAAAGCGATTAT-HCT116_Batch1,4.167945,True,2,1087.0,26080.0,HCT116_Batch1,5931,KCNK7_P1P2-1|KCNK7_P1P2-2,ENSG00000173338,KCNK7,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HCT116_filtered_dual_g..."
AAACCAAAGGCTTAAT-HCT116_Batch1,2.489183,True,2,955.0,38366.0,HCT116_Batch1,7157,APOA4_P1P2-1|APOA4_P1P2-2,ENSG00000110244,APOA4,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HCT116_filtered_dual_g..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTTGTGCAGGGAATCG-HCT116_Batch109,4.289229,True,2,767.0,17882.0,HCT116_Batch109,5107,ACP6_P1P2-1|ACP6_P1P2-2,ENSG00000162836,ACP6,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HCT116_filtered_dual_g..."
GTTGTGGGTCACGCGG-HCT116_Batch109,8.142244,True,2,664.0,8155.0,HCT116_Batch109,3643,RNF39_P1P2-1|RNF39_P1P2-2,ENSG00000204618,RNF39,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HCT116_filtered_dual_g..."
GTTGTGGGTCTGACTC-HCT116_Batch109,24.518473,True,2,1553.0,6334.0,HCT116_Batch109,2831,AFP_P2-1|AFP_P2-2,ENSG00000081051,AFP,...,barcode sequencing,None,CellRanger,None,custom,None,GRCh38,CC BY-NC-SA 4.0,SWO:1000090,"[{""dataset_accession"": ""HCT116_filtered_dual_g..."


### Curate replicate information

In [ ]:
cur_data.adata.obs['technical_replicate'] = cur_data.adata.obs['sample'].astype(str)

### Curate tissue information

In [ ]:

cur_data.standardize_ontology(
    input_column='tissue',
    column_type='term_name',
    ontology_type='tissue',
    overwrite=True
)

Mapped 1 tissue ontology terms from `tissue` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
  input_column input_column_lower name_lower     ontology_id
0        colon              colon      colon  UBERON:0001155
--------------------------------------------------


/usr/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


### Curate cell type information

In [ ]:
cur_data.standardize_ontology(
    input_column='cell_type',
    column_type='term_name',
    ontology_type='cell_type'
)

Mapped 1 cell_type ontology terms from `cell_type` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
            input_column     input_column_lower             name_lower  \
0  colon epithelial cell  colon epithelial cell  colon epithelial cell   

  ontology_id  
0  CL:0011108  
--------------------------------------------------


/usr/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


### Curate cell line information

In [ ]:
cur_data.standardize_ontology(
    input_column='cell_line',
    column_type='term_name',
    ontology_type='cell_line'
)

Mapped 1 cell_line ontology terms from `cell_line` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
   input_column input_column_lower    name_lower  ontology_id
0  HCT 116 cell       hct 116 cell  hct 116 cell  CLO:0003665
--------------------------------------------------


/usr/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


### Curate disease information

In [ ]:
cur_data.standardize_ontology(
    input_column='disease',
    column_type='term_name',
    ontology_type='disease'
)

Mapped 1 disease ontology terms from `disease` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
      input_column input_column_lower       name_lower    ontology_id
0  colon carcinoma    colon carcinoma  colon carcinoma  MONDO:0002032
--------------------------------------------------


/usr/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


### Match schema column order

In [ ]:
cur_data.match_schema_columns(slot='obs')

Matched columns of adata.obs to the obs_schema.


### Validate obs metadata

In [ ]:
sch = ObsSchema.to_schema()

for col in list(sch.columns.keys()):
    sch_subset = sch.select_columns([col])
    try:
        sch_subset.validate(cur_data.adata.obs[[col]], lazy=True)
        print(f"Validation successful for column {col}.")
    except Exception as e:
        print(f"Validation failed for column {col}: {e}")

Validation successful for column dataset_id.
Validation successful for column sample_id.
Validation successful for column data_modality.
Validation successful for column significant.
Validation successful for column significance_criteria.
Validation successful for column perturbation_name.
Validation successful for column perturbed_target_coord.
Validation successful for column perturbed_target_chromosome.
Validation successful for column perturbed_target_chromosome_encoding.
Validation successful for column perturbed_target_number.
Validation successful for column perturbed_target_ensg.
Validation successful for column perturbed_target_symbol.
Validation successful for column perturbed_target_biotype.
Validation successful for column guide_sequence.
Validation successful for column perturbation_type_label.
Validation successful for column perturbation_type_id.
Validation successful for column timepoint.
Validation successful for column treatment_label.
Validation successful for column

In [ ]:
# cur_data.validate_data(slot='obs')

# VAR slot curation

### Standardise genes

In [ ]:
cur_data.show_var()

Variable data:
DataFrame shape: (38606, 6)
--------------------------------------------------
                    mt  n_cells_by_counts   mean_counts  \
DDX11L2          False               2719  2.777434e-04   
MIR1302-2HG      False               2620  2.673255e-04   
FAM138A          False                  3  3.034341e-07   
ENSG00000290826  False                764  8.698444e-05   
OR4F5            False                263  2.700564e-05   
...                ...                ...           ...   
ENSG00000277836  False                  0  0.000000e+00   
ENSG00000278633  False                106  1.092363e-05   
ENSG00000276017  False                663  6.756467e-05   
ENSG00000278817  False              35369  3.616227e-03   
ENSG00000277196  False              48372  5.025274e-03   

                 pct_dropout_by_counts  total_counts      gene_symbol  
DDX11L2                      99.972499        2746.0          DDX11L2  
MIR1302-2HG                  99.973500        2643.0 

In [ ]:
cur_data.adata.var['ensembl_inputs'] = [e if e.startswith('ENSG') else None for e in cur_data.adata.var.index]
cur_data.adata.var['gene_inputs'] = [e if not e.startswith('ENSG') else '' for e in cur_data.adata.var.index]

var_df = cur_data.adata.var.copy()

In [ ]:
cur_data.create_columns(slot='var', col_dict={
    'genes':cur_data.adata.var.index
})

Column genes added to adata.var


In [ ]:
cur_data.standardize_genes(
    slot="var", input_column="gene_inputs", input_column_type="gene_symbol"
)
var_symb = cur_data.adata.var.copy()

Mapping gene symbols: 100%|█████████████████████████████████| 25645/25645 [00:02<00:00, 9096.70it/s]


--------------------------------------------------
Successfully mapped 25586 out of 25645 gene symbols.
--------------------------------------------------
Couldn't map gene symbols: ['', 'ARHGEF19-AS1', 'LYST-AS1', 'LINC01115-1', 'LINC01119', 'LSP1P5-1', 'LINC01889', 'UPP2-IT1', 'LINC01238-1', 'ARF4-AS1', 'DENND6A-AS1', 'PRICKLE2-AS1-1', 'SLC41A3-AS1', 'IFT122P3', 'LINC00886', 'LINC02263', 'TEMN3-AS1', 'CTD-2350J17.1', 'LINC02119', 'LINC01033', 'CTC-338M12.4', 'MIR5689HG', 'RAET1E-AS1-1', 'LINC03021-1', 'LINC03023-1', 'TNFRSF10A-DT-1', 'LINC01605-1', 'NCRNA00250', 'PTGER4P2-CDK2AP2P2', 'LINC03025-1', 'LINC00484-1', 'LINC02913', 'DNAJC9-AS1-1', 'LINC02722', 'LINC02398', 'GPR84-AS1-1', 'LINC02399', 'SPATA13-1', 'SMIM2', 'SMIM2-IT1', 'PCDH9-AS4', 'HIF1A-AS1', 'HIF1A-AS3', 'VASH1-AS1', 'GOLGA8M-1', 'LINC02256-1', 'LINC01580', 'DDX11L16-1', 'NPIPA9-1', 'LINC02911', 'MKKS-1', 'MKKS-2', 'D21S2088E', 'LINC01422', 'ELFN2-1', 'ARMCX5-GPRASP2-1', 'PRY2', 'PRY', 'MAFIP']
--------------------------

In [ ]:
cur_data.standardize_genes(
    slot="var", input_column="ensembl_inputs", input_column_type="ensembl_gene_id"
)
var_ensg = cur_data.adata.var.copy()

Missing Ensembl IDs: ['ENSG00000272922', 'ENSG00000224481', 'ENSG00000259582', 'ENSG00000287139', 'ENSG00000251399', 'ENSG00000277559', 'ENSG00000276017', 'ENSG00000275636', 'ENSG00000286571', 'ENSG00000263331', 'ENSG00000290872', 'ENSG00000289531', 'ENSG00000203520', 'ENSG00000257962', 'ENSG00000241912', 'ENSG00000261386', 'ENSG00000287811', 'ENSG00000290709', 'ENSG00000258921', 'ENSG00000230649', 'ENSG00000286472', 'ENSG00000259274', 'ENSG00000225489', 'ENSG00000236412', 'ENSG00000259209', 'ENSG00000277630', 'ENSG00000286977', 'ENSG00000229454', 'ENSG00000290078', 'ENSG00000286633', 'ENSG00000264421', 'ENSG00000244260', 'ENSG00000249638', 'ENSG00000267105', 'ENSG00000255328', 'ENSG00000266588', 'ENSG00000286603', 'ENSG00000234405', 'ENSG00000258345', 'ENSG00000260163', 'ENSG00000250360', 'ENSG00000257454', 'ENSG00000232056', 'ENSG00000289448', 'ENSG00000267420', 'ENSG00000264790', 'ENSG00000267537', 'ENSG00000260511', 'ENSG00000287622', 'ENSG00000277423', 'ENSG00000288816', 'ENSG0000

# Compile final gene symbols and ensg ids from the two mappings

In [ ]:
var_df_clean = var_symb.merge(var_ensg, left_index=True, right_index=True, suffixes=('_symb', '_ensg'))
var_df_clean = var_df_clean.drop(columns=['gene_inputs_symb', 'gene_inputs_ensg', 'ensembl_inputs_symb', 'ensembl_inputs_ensg'])
var_df_clean['clean_gene_symbol'] = None
var_df_clean['clean_ensg_id'] = None

ensg_starts_mask = var_df_clean.index.str.startswith('ENSG')

# if original id starts with ENSG - assign ensg mapping
var_df_clean.loc[ensg_starts_mask, 'clean_gene_symbol'] = var_df_clean.loc[ensg_starts_mask, 'gene_symbol_ensg']
var_df_clean.loc[ensg_starts_mask, 'clean_ensg_id'] = var_df_clean.loc[ensg_starts_mask, 'ensembl_gene_id_ensg']

# if not, assign symb mapping
var_df_clean.loc[~ensg_starts_mask, 'clean_gene_symbol'] = var_df_clean.loc[~ensg_starts_mask, 'gene_symbol_symb']
var_df_clean.loc[~ensg_starts_mask, 'clean_ensg_id'] = var_df_clean.loc[~ensg_starts_mask, 'ensembl_gene_id_symb']

# replace missing ensg with
var_df_clean.loc[var_df_clean['clean_ensg_id'].isna() & ensg_starts_mask, 'clean_ensg_id'] = var_df_clean.loc[var_df_clean['clean_ensg_id'].isna() & ensg_starts_mask].index

var_df_clean = var_df_clean.rename(columns={'clean_gene_symbol':'gene_symbol', 'clean_ensg_id':'ensembl_gene_id'})[['ensembl_gene_id', 'gene_symbol']]

var_df_clean


,ensembl_gene_id,gene_symbol
index,,
DDX11L2,ENSG00000310539,DDX11L2
MIR1302-2HG,ENSG00000243485,MIR1302-2HG
FAM138A,ENSG00000237613,FAM138A
ENSG00000290826,ENSG00000290826,None
OR4F5,ENSG00000186092,OR4F5
...,...,...
ENSG00000277836,ENSG00000277836,NaN
ENSG00000278633,ENSG00000278633,NaN
ENSG00000276017,ENSG00000276017,NaN


In [ ]:
# assign fixed genes back to var
cur_data.adata.var = var_df_clean

In [ ]:
var_df_clean.loc[var_df_clean['ensembl_gene_id'].notna() & var_df_clean['ensembl_gene_id'].str.startswith('LRG')]

,ensembl_gene_id,gene_symbol
index,,


### Validate var metadata

In [ ]:
cur_data.validate_data(slot='var')

,ensembl_gene_id,gene_symbol
index,,
DDX11L2,ENSG00000310539,DDX11L2
MIR1302-2HG,ENSG00000243485,MIR1302-2HG
FAM138A,ENSG00000237613,FAM138A
ENSG00000290826,ENSG00000290826,None
OR4F5,ENSG00000186092,OR4F5
...,...,...
ENSG00000277836,ENSG00000277836,NaN
ENSG00000278633,ENSG00000278633,NaN
ENSG00000276017,ENSG00000276017,NaN


# Save the dataset

In [ ]:
cur_data.save_curated_data_h5ad()

/content/PerturbationCatalogue/data_exploration/curation_tools/curation_tools.py:327: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adata.obs = adata.obs.fillna(value=np.nan)


✅ Curated h5ad data saved to Perturbseq/curated/h5ad/orion_2025_hct116_curated.h5ad


In [ ]:
cur_data.save_curated_data_parquet(split_metadata=True, save_metadata_only=True)

✅ Metadata saved to Perturbseq/curated/parquet/orion_2025_hct116_curated_metadata.parquet


# Upload to BigQuery

In [ ]:
upload_parquet_to_bq(
    parquet_path='/content/PerturbationCatalogue/data_exploration/Perturbseq/curated/parquet/orion_2025_hct116_curated_metadata.parquet',
    bq_dataset_id='prj-ext-dev-pertcat-437314.perturb_seq',
    bq_table_name='metadata',
    key_columns=['dataset_id', 'sample_id'],
    verbose=True
)

Staging table: loading `.parquet` file /content/PerturbationCatalogue/data_exploration/Perturbseq/curated/parquet/orion_2025_hct116_curated_metadata.parquet to prj-ext-dev-pertcat-437314.perturb_seq.metadata_staging...
Staging table: loaded 3409169 rows to prj-ext-dev-pertcat-437314.perturb_seq.metadata_staging
Staging table: added ingested_at timestamp column to prj-ext-dev-pertcat-437314.perturb_seq.metadata_staging
Merge completed: staging → prj-ext-dev-pertcat-437314.perturb_seq.metadata with type-safe casting.
Staging table: deleted prj-ext-dev-pertcat-437314.perturb_seq.metadata_staging


# Upload to GC Storage

In [ ]:
!gcloud storage cp Perturbseq/curated/h5ad/orion_2025_hct116_curated.h5ad gs://perturbation-catalogue-lake/perturbseq/curated/

uploading large objects. If you would like to opt-out and instead
perform a normal upload, run:
`gcloud config set storage/parallel_composite_upload_enabled False`
If you would like to disable this warning, run:
`gcloud config set storage/parallel_composite_upload_enabled True`
Note that with parallel composite uploads, your object might be
uploaded as a composite object
(https://cloud.google.com/storage/docs/composite-objects), which means
that any user who downloads your object will need to use crc32c
checksums to verify data integrity. gcloud storage is capable of
computing crc32c checksums, but this might pose a problem for other
clients.

Copying file://Perturbseq/curated/h5ad/orion_2025_hct116_curated.h5ad to gs://perturbation-catalogue-lake/perturbseq/curated/orion_2025_hct116_curated.h5ad
